<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Install packages

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts", "torchtext", "datasets", "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

In [ ]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [1]:
!git -C dl_hub pull


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 352 bytes | 352.00 KiB/s, done.
From https://github.com/jyanivaddi/dl_hub
   59ba79b..cd298c7  main       -> origin/main
Updating 59ba79b..cd298c7
Fast-forward
 Transformer/PL_main.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [2]:
import torch
import torch.nn as nn
#from dl_hub.Transformer.config import get_config
from config import get_config
from train import train_model
from PL_data_module import OpusDataModule
from PL_model import LitTransformer
from PL_main import train_transformer


In [3]:
cfg = get_config()
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 10


**Define data module**

In [4]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()

**Define PL Model**

In [5]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt)
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)


/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss_criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_criterion'])`.
  rank_zero_warn(


In [6]:
train_transformer(transformer_model, opus_dm, cfg, epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 75.1 M
----------------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/e183534/ERA_V1/venv

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: "No, Bessie; I have only just finished dusting."
    TARGET: — No, Bessie, ho terminato di spolverare.
 PREDICTED: — Ma che non è un ’ altra .
*****************************************
*****************************************
    SOURCE: His eyes were clear and no longer feverish, but their expression was stern.
    TARGET: Gli occhi erano limpidi, non più febbrili, ma la loro espressione era dura.
 PREDICTED: Il suo suo suo suo suo suo suo suo suo suo suo suo suo suo suo suo suo suo era stato .
*****************************************
*****************************************
    SOURCE: But he could think of nothing else to say. 'Bear to the right, sir, there's a stump there,' said the coachman, taking hold of the rein.
    TARGET: — Vi prego di prendere a destra, altrimenti troverete un ceppo — disse il cocchiere, correggendo Levin nella guida.
 PREDICTED: Ma non è un ’ ic , ma non è un ’ ic , e che non è un ’ ic , e si , e si 

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Harris was for chancing it.
    TARGET: Harris fu d’opinione d’affrontare il rischio.
 PREDICTED: Harris doveva .
*****************************************

*****************************************
    SOURCE: "Farewell, Miss Eyre, for the present; is that all?"
    TARGET: — State bene, signorina Eyre; nient'altro?
 PREDICTED: — Addio , signorina Eyre ; perché tutto questo è qui ?
*****************************************
*****************************************
    SOURCE: That is its chief - one may say, its only recommendation.
    TARGET: Questo è il principale e — si può dire — suo unico pregio.
 PREDICTED: È la cosa più grande : la sua lo può dire .
*****************************************

*****************************************
    SOURCE: You consider Vronsky an aristocrat. I don't.
    TARGET: Tu consideri Vronskij un aristocratico, ma io no.
 PREDICTED: Voi capite : “ Vronskij non .
*********************************

`Trainer.fit` stopped: `max_epochs=15` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

*****************************************
    SOURCE: You say it's hot' she asked, stopping at the door and looking inquiringly at Vronsky, while a bright flush again suffused her face.
    TARGET: Cosa dite, fa caldo? — disse, fermandosi sulla porta e guardando interrogativamente Vronskij. E di nuovo un vivace rossore le coprì il viso.
 PREDICTED: Voi dite : “ tu sei caldo , e , guardando la porta , guardò Vronskij con un viso grazioso e cupo , scomparve dietro a lei .
*****************************************
*****************************************
    SOURCE: He took his eldest son to visit the second family, and told Oblonsky that he considered it developed his son and was good for him.
    TARGET: E portava il primogenito nella seconda famiglia e raccontava a Stepan Arkad’ic che riteneva ciò utile e adatto a sviluppare il figlio.
 PREDICTED: il figlio più di Stepan Arkad ’ ic , il secondo viaggio , Stepan Arkad ’ ic raccontò che gli era stato carattere del figlio e lo aveva per 

In [ ]:
!tensorboard --logdir=runs